# Defining the fitness and criticality function


To define a new fitness function we need to implement the `Fitness` class (interface). 
The fitness class provides the `eval` function which needs to be implemented with concrete evaluation instructions. The `eval`function receives as input an instance of  [`SimulationOutput`](https://git.fortiss.org/opensbt/opensbt-core/-/blob/main/simulation/simulator.py#L40-62) and can return a scalar (one objective) or a vector valued output (several objectives). Further it contains the `min_max` function which has to return a tuple of `min` or `max` strings indicating whether the corresponding objective function is minimized or maximized.

In our example below we specify the fitness function with the objectives (taken from the CARLA example in [](/doc/jupyter/02_Example_CARLA.ipynb)): 1) minimize the distance to the adversary actors, 2) maximize the velocity of ego at the time of the minimal distance.


```python
class FitnessMinDistanceVelocity(Fitness):
    @property
    def min_or_max(self):
        return "min", "max"

    @property
    def name(self):
        return "Min distance", "Velocity at min distance"

    def eval(self, simout: SimulationOutput) -> Tuple[float]:
        traceEgo = simout.location["ego"]                     # The actor names have to match the names written into the simulationoutput instance by the simulation adapter
        tracePed = simout.location["adversary"]              

        ind_min_dist = np.argmin(geometric.distPair(traceEgo, tracePed))

        # distance between ego and other object
        distance = np.min(geometric.distPair(traceEgo, tracePed))

        # speed of ego at time of the minimal distance
        speed = simout.speed["ego"][ind_min_dist]

        return (distance, speed)

```


When testing a system, in general, some understanding exists for which fitness values a test input is considered as failure-revealing or not. In the context of ADS testing this notion is defined by the safety requirement, telling that the ego vehicle shall maintain a safety distance of .5m.

In OpenSBT, such safety requirements can be represented by implementing the interface class [Critical](evaluation/critical.py). `Critical` offers the method `eval`, where specific constraints can be implemented. It received as input the fitness vector in form of an array, and the simulation output for the corresponding test input.

In the example below the Critical Function indicates that when the distance is below .5m and ego is moving with positive velocity then the test input is critical. Consider that the sign in the inequations is inverted for the second constraint, as the velocity is to be maximized.


```python
class CriticalExample(Critical):
    def eval(self, vector_fitness, simout: SimulationOutput = None):
        if (vector_fitness[0] < 0.5) and (vector_fitness[1] < 0):
            return True
        else:
            return False
```